In [ ]:
import os
os.chdir("..")
print(os.getcwd())


In [ ]:
%load_ext autoreload
%load_ext line_profiler

In [ ]:
%matplotlib qt
%autoreload 2

In [ ]:
from retna.networks import *
from retna.training import *
from retna.data_loader import *
from retna.view import *
from retna.prune import *
from retna.hook_tools import *
from retna.simulate_data import *

In [ ]:
import torch
import glob as glob
import matplotlib.pyplot as plt

## Load Model

In [ ]:
h_chans = [30,30,20,20,20,20]
model = Retna_V1(1,6, h_chans)

In [ ]:
SimLoader = SimDataset(5)

In [ ]:
x,y = next(iter(SimLoader))

In [ ]:
plt.figure()
plt.imshow(x[0,0])
plt.figure()
plt.imshow(y[0,2], cmap="gray")

In [ ]:
train_model_x( model, SimLoader, num_epochs=50 )

In [ ]:
%lprun -f print_mosaic_x print_mosaic_x(SimLoader,model,  n=5)

In [ ]:
print_comparisons_x(model,SimLoader)

In [ ]:
print_mosaic_x(SimLoader,model,  n=5)

## Load Simulated Data into H5 Format

In [ ]:
image, mask = generate_random_data(120, 120, count=500)
#image = image.transpose(0,3,1,2)[:,0:1]

In [ ]:
plt.figure()
plt.imshow(image[0,0])
plt.figure()
plt.imshow(mask[0,2])

In [ ]:
root = ""  ## Add desination folder here
filename = root + "example_data.h5"

In [ ]:
import h5py 
with h5py.File(filename, 'w') as fh:
    dset = fh.create_dataset("input", data=image , compression="lzf" )
    dset = fh.create_dataset("target", data=mask , compression="lzf" )

## Load Data from H5 Format

In [ ]:
data = H5_Handler([filename], datasets=["input","target"])
Loader = DataLoader(data)

In [ ]:
## Specify what header the target data is in. 
Loader.dataset.dataset_names[1]= "target"  
Loader.dataset.channels[1] = None

In [ ]:
## Specify what header the target data is in. 

In [ ]:
model = model.to("cpu")
Loader.dataset.device = "cpu"

In [ ]:
train_model( model, Loader, num_epochs=10 )

In [ ]:
print_comparisons(model,Loader)

In [ ]:
print_mosaic(Loader,model,m=5,n=10)

## Load Data from Tiff Format

In [ ]:
#wip

## Load MNIST EXAMPLE

In [ ]:
import torch
import torchvision.datasets as datasets 
from torchvision.transforms import transforms

In [ ]:
mnist = datasets.MNIST(root='./data', train=True, download=False, transform=transforms.ToTensor())
loader =torch.utils.data.DataLoader(dataset=mnist, batch_size=100, shuffle=True)

In [ ]:
x,y = next(iter(loader))
plt.imshow(x[15,0])

In [ ]:
h_chans = [30,30,30,20,20,20]
model = Retna_V1(1,10, h_chans)

In [ ]:
def onehot(y):
  truth = np.zeros((len(y),10))
  for n in range(len(truth)):
    truth[n,y[n]] = 1

  truth = torch.Tensor(truth)
  truth = truth[...,None,None]

  return truth


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for e in range(1000):

  optimizer.zero_grad()
  x,y = next(iter(loader))
  pred = model(x)
  pred = pred.mean(axis=(2,3), keepdim=True)
  truth = onehot(y)
  loss = dice_loss(pred,truth).mean()
  loss.backward()
  optimizer.step()

  print(loss.detach())

In [ ]:
pred = model(x)
pred = pred.mean(axis=(2,3), keepdim=True)

In [278]:
  x,y = next(iter(loader))
  pred = model(x)
  pred = pred.mean(axis=(2,3), keepdim=True)
  truth = onehot(y)
  print(np.argmax(pred.detach(),axis=1)[:,0,0])
  print(y)

tensor([1, 5, 6, 3, 3, 3, 2, 9, 8, 3, 2, 9, 9, 5, 9, 6, 7, 4, 0, 0, 2, 4, 4, 0,
        9, 0, 5, 6, 6, 4, 0, 5, 4, 3, 2, 4, 2, 2, 0, 6, 1, 7, 5, 4, 9, 7, 9, 1,
        5, 1, 5, 8, 6, 2, 9, 4, 5, 4, 5, 8, 4, 4, 3, 9, 1, 1, 0, 1, 1, 6, 3, 7,
        6, 9, 5, 2, 1, 3, 1, 0, 9, 6, 7, 6, 9, 4, 0, 7, 1, 2, 3, 8, 0, 7, 4, 5,
        5, 4, 0, 6])
tensor([1, 5, 6, 3, 7, 3, 2, 9, 8, 9, 2, 9, 9, 5, 9, 6, 7, 4, 0, 0, 2, 4, 4, 0,
        9, 0, 5, 6, 6, 4, 0, 5, 4, 3, 2, 4, 2, 2, 0, 6, 1, 7, 5, 4, 9, 7, 9, 1,
        5, 1, 5, 8, 6, 2, 9, 4, 5, 4, 5, 8, 4, 4, 3, 9, 1, 1, 0, 1, 1, 6, 3, 7,
        6, 9, 5, 2, 1, 3, 1, 0, 9, 6, 7, 6, 9, 4, 0, 7, 1, 2, 3, 8, 0, 7, 4, 5,
        4, 4, 0, 6])


In [ ]:
plt.imshow(x[7,0])

In [ ]:
pred = model(x).mean(axis=(2,3))
pred[7]

In [ ]:
pred = model(x[[0]])
score = pred.mean(axis=(2,3))
score.shape

In [ ]:
plt.imshow(pred[0,2].detach().numpy())

In [ ]:
plt.imshow(pred[1,1].detach().numpy())

In [ ]:
class SimNumber(Dataset):
    def __init__(self, count=10, transform=None):
        self.count = count
        #self.input_images, self.target_masks = generate_random_data(120, 120, count=count)        
        self.transform = transform

        mnist = datasets.MNIST(root='./data', train=True, download=False, transform=transforms.ToTensor())
        loader =torch.utils.data.DataLoader(dataset=mnist, batch_size=1, shuffle=True)

        self.dataset = iter(loader)


    def __len__(self):
        return len(range(self.count))
    
    def __getitem__(self, idx): 
        range(self.count)[idx]


        image = np.zeros((30,100))
        mask  = np.zeros((9,30,100))


        for n in range(3):

            nx = n*28

            im, lb = next(self.dataset)

            image[:28, nx:nx+28]    = im
            mask[lb, :28, nx:nx+28] = im 
        

        if self.transform: pass
        #image = self.transform(image)


        image = torch.tensor(image).float()
        mask  = torch.tensor(mask).float()
        
        return [image, mask]

In [ ]:
iter_mnist = iter(loader)
x = next(iter_mnist)

In [ ]:
x,y = next(iter(SimNumber()))

In [ ]:
import numpy as np


In [ ]:
plt.imshow(x)

In [ ]:
plt.imshow(y[3])